In [2]:
import numpy as np
import pandas as pd
import pickle
import math

In [3]:
participants = pd.read_csv('../data_clean/2023-01-28_participants_clean_an-exp1_n-91.csv', sep = ",")
transitions = pd.read_csv("../data_clean/2023-01-28_transitions_clean_an-exp1_n-91.csv", sep = ",")
trials_learning = pd.read_csv('../data_clean/2023-01-28_learning_clean_an-exp1_n-91.csv', sep = ",")
trials_prediction = pd.read_csv('../data_clean/2023-01-28_test_prediction_clean_an-exp1_n-91.csv', sep = ",")
trials_control = pd.read_csv('../data_clean/2023-01-28_test_control_clean_an-exp1_n-91.csv', sep = ",")
trials_explanation = pd.read_csv('../data_clean/2023-01-28_test_explanation_clean_an-exp1_n-91.csv', sep = ",")

In [4]:
# Define functions for hidden cases

def predict_normative(start_state, first_input, second_input, fsm_temp):
    """Return an array with probabilities of each final state"""
    states = [0, 1, 2, 3]
    return np.array(
        [np.array([fsm_temp[start_state][first_input][middle_state] * fsm_temp[middle_state][second_input][final_state]
                   for middle_state in states]).sum()
         for final_state in states],
        dtype=[(str(start_state) + "—" + first_input + "–[All paths]–" + second_input + "–?", "float16")])

def predict_an(start_state, first_input, second_input, fsm_temp):
    states = [0, 1, 2, 3]
    max_middle_state = np.array([fsm_temp[start_state][first_input]]).argmax()
    ans = np.array([np.array([fsm_temp[middle_state][second_input][final_state]
                              for middle_state in [max_middle_state]]).sum()
                    for final_state in states])
    return np.array(ans / ans.sum(),
                    dtype=[(str(start_state) + "—" + first_input + "–[ML path]–" + second_input + "–?", "float16")])

def control_normative(start_state, final_state, fsm_temp):
    """ Calculate probabilities of each possible combination of inputs for every possible middle state and return a dictionary"""
    states = [0, 1, 2, 3]
    input_combinations = [["a", "a"], ["a", "b"], ["b", "a"], ["b", "b"]]
    control_all_middle_states = {}

    for inp in input_combinations:
        label = inp[0] + inp[1]
        control_all_middle_states[label] = np.array([fsm_temp[start_state][inp[0]][middle_state] *
                                                     fsm_temp[middle_state][inp[1]][final_state] for middle_state in states]).sum()
    return control_all_middle_states

#control_normative(1, 0, fsm)["ba"]

def control_an(start_state, final_state, fsm_temp):
    """ Calculate probabilities of each possible combination of inputs for every possible middle state and return a dictionary"""
    input_combinations = [["a", "a"], ["a", "b"], ["b", "a"], ["b", "b"]]
    control_ml_middle_state = {}
    for inp in input_combinations:
        label = inp[0] + inp[1]
        max_middle_state = np.array([fsm_temp[start_state][inp[0]]]).argmax()
        control_ml_middle_state[label] = np.array([fsm_temp[middle_state][inp[1]][final_state] for middle_state in [max_middle_state]]).sum()
    return control_ml_middle_state

# Explanation hidden
def explain_normative(start_state, first_input, second_input, final_state, fsm_temp):
    ans = {}
    states = [0, 1, 2, 3]

    # first input counterfactual
    first_input_cf = "b" if first_input == "a" else "a" #counteractual input 1
    explanation_1 = np.array([fsm_temp[start_state][first_input_cf][middle_state] * fsm_temp[middle_state][second_input][final_state]
                                                  for middle_state in states]).sum()
    ans["1"] = explanation_1

    # second input counterfactual
    second_input_cf = "b" if second_input == "a" else "a" #counteractual input 2
    explanation_2 = np.array([fsm_temp[start_state][first_input][middle_state] * fsm_temp[middle_state][second_input_cf][final_state]
                                                  for middle_state in states]).sum()
    ans["2"] = explanation_2

    return ans

def explain_an(start_state, first_input, second_input, final_state, fsm_temp):
    ans = {}

    # first input counterfactual
    first_input_cf = "b" if first_input == "a" else "a" # counteractual input 1
    max_middle_state1 = np.array([fsm_temp[start_state][first_input_cf]]).argmax()
    explanation_1 = np.array([fsm_temp[middle_state][second_input][final_state] for middle_state in [max_middle_state1]]).sum()
    ans["1"] = explanation_1

    # second input counterfactual
    second_input_cf = "b" if second_input == "a" else "a" # counteractual input 2
    max_middle_state2 = np.array([fsm_temp[start_state][first_input]]).argmax()
    explanation_2 = np.array([fsm_temp[middle_state][second_input_cf][final_state] for middle_state in [max_middle_state2]]).sum()
    ans["2"] = explanation_2

    return ans

In [5]:
def compute_for_trials(trials, task, form, epsilon, beta):
    import math
    log_p_answers = []
    for index in range(0, trials.shape[0]):
        option_1_p_mm = trials.iloc[[index]].option_1_p_mm.item()
        option_2_p_mm = trials.iloc[[index]].option_2_p_mm.item()
        response = trials.iloc[[index]].response.item()

        if task == "prediction":
            option_1 = int(trials.iloc[[index]].option_1.item())
            option_2 = int(trials.iloc[[index]].option_2.item())
        if task == "explanation":
            option_1 = 1
            option_2 = 2
        if task == "control":
            option_1 = trials.iloc[[index]].option_1.item() if form == "hidden" else "a"
            option_2 = trials.iloc[[index]].option_2.item() if form == "hidden" else "b"

        # R_i
        if option_1_p_mm > option_2_p_mm:
            R_i = math.log((option_1_p_mm + epsilon)/(option_2_p_mm + epsilon))
        elif option_1_p_mm < option_2_p_mm:
            R_i = math.log((option_2_p_mm + epsilon)/(option_1_p_mm + epsilon))
        else:
            R_i = 1

        p_correct = (math.e ** (R_i * beta)) / (math.e ** (R_i * beta) + math.e ** (-1 * R_i * beta))

        if task == "explanation":
            correct_mm = option_1 if option_1_p_mm < option_2_p_mm else option_2
        else: correct_mm = option_1 if option_1_p_mm > option_2_p_mm else option_2

        p_answer = p_correct if response == correct_mm else (1 - p_correct)
        log_p_answer = math.log(p_answer)
        log_p_answers.append(log_p_answer)
    return log_p_answers

def compute_for_trials_an(trials, task, form, epsilon, beta):
    import math
    log_p_answers = []
    for index in range(0, trials.shape[0]):
        option_1_p_mm = trials.iloc[[index]].option_1_p_mm_an.item()
        option_2_p_mm = trials.iloc[[index]].option_2_p_mm_an.item()
        response = trials.iloc[[index]].response.item()

        if task == "prediction":
            option_1 = int(trials.iloc[[index]].option_1.item())
            option_2 = int(trials.iloc[[index]].option_2.item())
        if task == "explanation":
            option_1 = 1
            option_2 = 2
        if task == "control":
            option_1 = trials.iloc[[index]].option_1.item() if form == "hidden" else "a"
            option_2 = trials.iloc[[index]].option_2.item() if form == "hidden" else "b"

        # R_i
        if option_1_p_mm > option_2_p_mm:
            R_i = math.log((option_1_p_mm + epsilon)/(option_2_p_mm + epsilon))
        elif option_1_p_mm < option_2_p_mm:
            R_i = math.log((option_2_p_mm + epsilon)/(option_1_p_mm + epsilon))
        else:
            R_i = 1

        p_correct = (math.e ** (R_i * beta)) / (math.e ** (R_i * beta) + math.e ** (-1 * R_i * beta))

        if task == "explanation":
            correct_mm = option_1 if option_1_p_mm < option_2_p_mm else option_2
        else: correct_mm = option_1 if option_1_p_mm > option_2_p_mm else option_2

        p_answer = p_correct if response == correct_mm else (1 - p_correct)
        log_p_answer = math.log(p_answer)
        log_p_answers.append(log_p_answer)
    return log_p_answers

def compute_for_trials_fsm(trials, task, form, epsilon, beta):
    import math
    log_p_answers = []
    for index in range(0, trials.shape[0]):
        option_1_p_mm = trials.iloc[[index]].option_1_p.item()
        option_2_p_mm = trials.iloc[[index]].option_2_p.item()
        response = trials.iloc[[index]].response.item()

        if task == "prediction":
            option_1 = int(trials.iloc[[index]].option_1.item())
            option_2 = int(trials.iloc[[index]].option_2.item())
        if task == "explanation":
            option_1 = 1
            option_2 = 2
        if task == "control":
            option_1 = trials.iloc[[index]].option_1.item() if form == "hidden" else "a"
            option_2 = trials.iloc[[index]].option_2.item() if form == "hidden" else "b"

        # R_i
        if option_1_p_mm > option_2_p_mm:
            R_i = math.log((option_1_p_mm + epsilon)/(option_2_p_mm + epsilon))
        elif option_1_p_mm < option_2_p_mm:
            R_i = math.log((option_2_p_mm + epsilon)/(option_1_p_mm + epsilon))
        else:
            R_i = 1

        p_correct = (math.e ** (R_i * beta)) / (math.e ** (R_i * beta) + math.e ** (-1 * R_i * beta))

        if task == "explanation":
            correct_mm = option_1 if option_1_p_mm < option_2_p_mm else option_2
        else: correct_mm = option_1 if option_1_p_mm > option_2_p_mm else option_2

        p_answer = p_correct if response == correct_mm else (1 - p_correct)
        log_p_answer = math.log(p_answer)
        log_p_answers.append(log_p_answer)
    return log_p_answers

def compute_for_trials_fsm_an(trials, task, form, epsilon, beta):
    import math
    log_p_answers = []
    for index in range(0, trials.shape[0]):
        option_1_p_mm = trials.iloc[[index]].option_1_p_an.item()
        option_2_p_mm = trials.iloc[[index]].option_2_p_an.item()
        response = trials.iloc[[index]].response.item()

        if task == "prediction":
            option_1 = int(trials.iloc[[index]].option_1.item())
            option_2 = int(trials.iloc[[index]].option_2.item())
        if task == "explanation":
            option_1 = 1
            option_2 = 2
        if task == "control":
            option_1 = trials.iloc[[index]].option_1.item() if form == "hidden" else "a"
            option_2 = trials.iloc[[index]].option_2.item() if form == "hidden" else "b"

        # R_i
        if option_1_p_mm > option_2_p_mm:
            R_i = math.log((option_1_p_mm + epsilon)/(option_2_p_mm + epsilon))
        elif option_1_p_mm < option_2_p_mm:
            R_i = math.log((option_2_p_mm + epsilon)/(option_1_p_mm + epsilon))
        else:
            R_i = 1

        p_correct = (math.e ** (R_i * beta)) / (math.e ** (R_i * beta) + math.e ** (-1 * R_i * beta))

        if task == "explanation":
            correct_mm = option_1 if option_1_p_mm < option_2_p_mm else option_2
        else: correct_mm = option_1 if option_1_p_mm > option_2_p_mm else option_2

        p_answer = p_correct if response == correct_mm else (1 - p_correct)
        log_p_answer = math.log(p_answer)
        log_p_answers.append(log_p_answer)
    return log_p_answers

In [95]:
# Add participants' data in a dictionary + compute MM probabilities

sample = {} # created empt dict for all data
eps = 0.01

betas = np.arange(0, 1, 0.01)

fsms = {'easy': [{'a': np.array([0, 0, 1, 0]), 'b': np.array([0, 0, 1, 0])},  # state 0
                 {'a': np.array([1, 0, 0, 0]), 'b': np.array([1, 0, 0, 0])},  # state 1
                 {'a': np.array([0.6, 0, 0, 0.4]), 'b': np.array([0, 0, 1, 0])},  # state 2
                 {'a': np.array([0.4, 0.6, 0, 0]), 'b': np.array([0, 0, 0, 1])}  # state 3
                 ],
        'hard': [{'a': np.array([0, 0, 0, 1]), 'b': np.array([0, 0, 0, 1])},  # state 0
                 {'a': np.array([0.4, 0.6, 0, 0]), 'b': np.array([0, 0.6, 0, 0.4])},  # state 1
                 {'a': np.array([0, 0, 1, 0]), 'b': np.array([0.4, 0.6, 0, 0])},  # state 2
                 {'a': np.array([0.4, 0, 0.6, 0]), 'b': np.array([0, 0, 0.6, 0.4])}  # state 3
                 ]}

for participant_id in trials_learning.participant_id.unique():
    # Import data for one subject
    ## get data for a single participant
    mm = transitions[transitions['participant_id'] == participant_id] # get metal model as a DataFrame
    fsm = participants.study_fsm[participants.participant_id == participant_id].item()
    learning = trials_learning[trials_learning.participant_id == participant_id]

    # Get Visible and Hidden trials for P, E, and C
    if trials_prediction[trials_prediction.participant_id == participant_id].size > 0:
        trials_vis = trials_prediction[(trials_prediction.participant_id == participant_id) &
                                    (trials_prediction.trial_type == "visible")]
        trials_hid = trials_prediction[(trials_prediction.participant_id == participant_id) &
                                   (trials_prediction.trial_type == "hidden")]
        task = "prediction"

    if trials_control[trials_control.participant_id == participant_id].size > 0:
        trials_vis = trials_control[(trials_control.participant_id == participant_id) &
                                    (trials_control.trial_type == "visible")]
        trials_hid = trials_control[(trials_control.participant_id == participant_id) &
                                   (trials_control.trial_type == "hidden")]
        task = "control"

    if trials_explanation[trials_explanation.participant_id == participant_id].size > 0:
        trials_vis = trials_explanation[(trials_explanation.participant_id == participant_id) &
                                 (trials_explanation.trial_type == "visible")]
        trials_hid = trials_explanation[(trials_explanation.participant_id == participant_id) &
                                (trials_explanation.trial_type == "hidden")]
        task = "explanation"

    ## select only informative columns
    trials_vis = trials_vis.iloc[:,4:]
    trials_hid = trials_hid.iloc[:,4:]

    # reformat mental model as dictionary
    MM = {}
    for state_current in mm.state_current.unique():
        inputs = {}
        for inp in ["a", "b"]:
            inputs[inp] = np.array([mm.state_next_p[(mm.state_current == state_current) & (mm.response_current == inp) & (mm.state_next == 0)].item(),
                             mm.state_next_p[(mm.state_current == state_current) & (mm.response_current == inp) & (mm.state_next == 1)].item(),
                             mm.state_next_p[(mm.state_current == state_current) & (mm.response_current == inp) & (mm.state_next == 2)].item(),
                             mm.state_next_p[(mm.state_current == state_current) & (mm.response_current == inp) & (mm.state_next == 3)].item()])
        MM[state_current] = {"a":inputs["a"], "b": inputs["b"]}


    # Use MM to calculate correct answers
    if task == "prediction":
        # Visible
        for index in range(0, trials_vis.shape[0]): # shape[0] = number of rows
            state_2 = int(trials_vis.iloc[[index]].state_2.item())
            response_2 = trials_vis.iloc[[index]].response_2.item()
            option_1 = int(trials_vis.iloc[[index]].option_1.item())
            option_2 = int(trials_vis.iloc[[index]].option_2.item())
            trials_vis.loc[trials_vis.index[index], 'option_1_p_mm'] = MM[state_2][response_2][option_1]
            trials_vis.loc[trials_vis.index[index], 'option_2_p_mm'] = MM[state_2][response_2][option_2]

        # Hidden
        for index in range(0, trials_hid.shape[0]):
            state_1 = int(trials_hid.iloc[[index]].state_1.item())
            response_1 = trials_hid.iloc[[index]].response_1.item()
            response_2 = trials_hid.iloc[[index]].response_2.item()
            option_1 = int(trials_hid.iloc[[index]].option_1.item())
            option_2 = int(trials_hid.iloc[[index]].option_2.item())

            trials_hid.loc[trials_hid.index[index], 'option_1_p_mm'] = predict_normative(state_1, response_1, response_2, MM)[option_1][0]
            trials_hid.loc[trials_hid.index[index], 'option_2_p_mm'] = predict_normative(state_1, response_1, response_2, MM)[option_2][0]

            trials_hid.loc[trials_hid.index[index], 'option_1_p_mm_an'] = predict_an(state_1, response_1, response_2, MM)[option_1][0]
            trials_hid.loc[trials_hid.index[index], 'option_2_p_mm_an'] = predict_an(state_1, response_1, response_2, MM)[option_2][0]

            this_fsm = fsms[fsm]
            trials_hid.loc[trials_hid.index[index], 'option_1_p_an'] = predict_an(state_1, response_1, response_2, this_fsm)[option_1][0]
            trials_hid.loc[trials_hid.index[index], 'option_2_p_an'] = predict_an(state_1, response_1, response_2, this_fsm)[option_2][0]

    if task == "control":
        # Visible
        for index in range(0, trials_vis.shape[0]): # shape[0] = number of rows
            state_2 = int(trials_vis.iloc[[index]].state_2.item())
            state_3 = int(trials_vis.iloc[[index]].state_3.item())
            option_1 = "a"
            option_2 = "b"
            trials_vis.loc[trials_vis.index[index], 'option_1_p_mm'] = MM[state_2][option_1][state_3]
            trials_vis.loc[trials_vis.index[index], 'option_2_p_mm'] = MM[state_2][option_2][state_3]

        # Hidden
        for index in range(0, trials_hid.shape[0]):
            state_1 = int(trials_hid.iloc[[index]].state_1.item())
            state_3 = int(trials_hid.iloc[[index]].state_3.item())
            option_1 = trials_hid.iloc[[index]].option_1.item()
            option_2 = trials_hid.iloc[[index]].option_2.item()

            trials_hid.loc[trials_hid.index[index], 'option_1_p_mm'] = control_normative(state_1, state_3, MM)[option_1]
            trials_hid.loc[trials_hid.index[index], 'option_2_p_mm'] = control_normative(state_1, state_3, MM)[option_2]

            trials_hid.loc[trials_hid.index[index], 'option_1_p_mm_an'] = control_an(state_1, state_3, MM)[option_1]
            trials_hid.loc[trials_hid.index[index], 'option_2_p_mm_an'] = control_an(state_1, state_3, MM)[option_2]

            this_fsm = fsms[fsm]
            trials_hid.loc[trials_hid.index[index], 'option_1_p'] = control_normative(state_1, state_3, this_fsm)[option_1]
            trials_hid.loc[trials_hid.index[index], 'option_2_p'] = control_normative(state_1, state_3, this_fsm)[option_2]
            trials_hid.loc[trials_hid.index[index], 'option_1_p_an'] = control_an(state_1, state_3, this_fsm)[option_1]
            trials_hid.loc[trials_hid.index[index], 'option_2_p_an'] = control_an(state_1, state_3, this_fsm)[option_2]

    if task == "explanation":
        # Visible
        for index in range(0, trials_vis.shape[0]): # shape[0] = number of rows
            state_1 = int(trials_vis.iloc[[index]].state_1.item())
            response_1 = trials_vis.iloc[[index]].response_1.item()
            state_2 = int(trials_vis.iloc[[index]].state_2.item())
            response_2 = trials_vis.iloc[[index]].response_2.item()
            state_3 = int(trials_vis.iloc[[index]].state_3.item())
            response_1_cf = "a" if response_1 == "b" else "b"
            response_2_cf = "a" if response_2 == "b" else "b"
            trials_vis.loc[trials_vis.index[index], 'option_1_p_mm'] = MM[state_1][response_1_cf][state_2] * MM[state_2][response_2][state_3]
            trials_vis.loc[trials_vis.index[index], 'option_2_p_mm'] = MM[state_1][response_1][state_2] * MM[state_2][response_2_cf][state_3]

            # Add actual probabilities (based on FSM, not MM)
            this_fsm = fsms[fsm]
            trials_vis.loc[trials_vis.index[index], 'option_1_p'] = this_fsm[state_1][response_1_cf][state_2] * this_fsm[state_2][response_2][state_3]
            trials_vis.loc[trials_vis.index[index], 'option_2_p'] = this_fsm[state_1][response_1][state_2] * this_fsm[state_2][response_2_cf][state_3]


        # Hidden
        for index in range(0, trials_hid.shape[0]): # shape[0] = number of rows
            state_1 = int(trials_hid.iloc[[index]].state_1.item())
            response_1 = trials_hid.iloc[[index]].response_1.item()
            response_2 = trials_hid.iloc[[index]].response_2.item()
            state_3 = int(trials_hid.iloc[[index]].state_3.item())
            response_1_cf = "a" if response_1 == "b" else "b"
            response_2_cf = "a" if response_2 == "b" else "b"
            trials_hid.loc[trials_hid.index[index], 'option_1_p_mm'] = explain_normative(state_1, response_1, response_2, state_3, MM)["1"]
            trials_hid.loc[trials_hid.index[index], 'option_2_p_mm'] = explain_normative(state_1, response_1, response_2, state_3, MM)["2"]


            trials_hid.loc[trials_hid.index[index], 'option_1_p_mm_an'] = explain_an(state_1, response_1, response_2, state_3, MM)["1"]
            trials_hid.loc[trials_hid.index[index], 'option_2_p_mm_an'] = explain_an(state_1, response_1, response_2, state_3, MM)["2"]

            this_fsm = fsms[fsm]
            trials_hid.loc[trials_hid.index[index], 'option_1_p'] = explain_normative(state_1, response_1, response_2, state_3, this_fsm)["1"]
            trials_hid.loc[trials_hid.index[index], 'option_2_p'] = explain_normative(state_1, response_1, response_2, state_3, this_fsm)["2"]
            trials_hid.loc[trials_hid.index[index], 'option_1_p_an'] = explain_an(state_1, response_1, response_2, state_3, this_fsm)["1"]
            trials_hid.loc[trials_hid.index[index], 'option_2_p_an'] = explain_an(state_1, response_1, response_2, state_3, this_fsm)["2"]

    # Add beta values
    beta_person = {}
    for form in ["visible", "hidden"]:
        log_p_answers = []
        if form == "visible":
            trials = trials_vis
        else: trials = trials_hid
        all_betas = []
        all_betas_an = []
        all_betas_fsm = []
        all_betas_fsm_an = []
        for beta in betas:
            log_p_answers = compute_for_trials(trials, task, form, eps, beta)
            all_betas.append([round(beta, 2), sum(log_p_answers)])
            # the same for current FSM, not MM
            log_p_answers_fsm = compute_for_trials_fsm(trials, task, form, eps, beta)
            all_betas_fsm.append([round(beta, 2), sum(log_p_answers_fsm)])
            if form == "hidden":
                log_p_answers_an = compute_for_trials_an(trials, task, form, eps, beta)
                all_betas_an.append([round(beta, 2), sum(log_p_answers_an)])

                # the same for current FSM, not MM
                log_p_answers_fsm_an = compute_for_trials_fsm_an(trials, task, form, eps, beta)
                all_betas_fsm_an.append([round(beta, 2), sum(log_p_answers_fsm_an)])
        beta_person[form + "_mm"] = all_betas
        beta_person[form + "_fsm"] = all_betas_fsm
    beta_person["hidden_an_mm"] = all_betas_an
    beta_person["hidden_an_fsm"] = all_betas_fsm_an

    max_betas = {}

    for condition in beta_person.keys():
        betas_task = np.array(beta_person[condition])[:,0]
        values = np.array(beta_person[condition])[:,1]
        max_beta = betas_task[values.argmax()]
        R_i = 3
        p_correct_r3 = (math.e ** (R_i * max_beta)) / (math.e ** (R_i * max_beta) + math.e ** (-1 * R_i * max_beta))
        max_betas[condition] = np.array([max_beta, p_correct_r3])

    ## record that data in a dictionary
    sample[participant_id] = {"task": task,
                              "fsm": fsm,
                              "learning": learning,
                              "MM": MM,
                              "visible": trials_vis,
                              "hidden": trials_hid,
                              "visible_beta": beta_person["visible_mm"],
                              "hidden_beta": beta_person["hidden_mm"],
                              "visible_beta_fsm": beta_person["visible_fsm"],
                              "hidden_beta_fsm": beta_person["hidden_fsm"],
                              "hidden_beta_fsm_an": beta_person["hidden_an_fsm"],
                              "hidden_beta_an": beta_person["hidden_an_mm"],
                              "max_betas": max_betas} # make it list by adding ".values.tolist()"

pickle.dump(sample, open( "sample.p", "wb" ) )

/Users/roman/opt/anaconda3/envs/workspace/lib/python3.7/site-packages/ipykernel_launcher.py:18: RuntimeWarning: invalid value encountered in true_divide


0.05

In [91]:
sample[1]["max_betas"][condition][0]
sample[1]["max_betas"][condition][1]

0.07

In [97]:
# export Expected Accuracy results for participants
results = pd.DataFrame(columns=['participant_id','fsm','task','condition','beta_max', 'p_correct_r3'])
this_row = {}
for participant_id in sample.keys():
    results_participant = pd.DataFrame(columns=['participant_id','fsm','task','condition','beta_max', 'p_correct_r3'])
    fsm = sample[participant_id]["fsm"]
    task = sample[participant_id]["task"]
    for condition in sample[participant_id]["max_betas"].keys():
        betas = sample[participant_id]["max_betas"][condition]
        this_row = {"participant_id": participant_id,
                    "fsm": fsm,
                    "task":task,
                    'condition':condition,
                    "beta_max": betas[0],
                    "p_correct_r3": betas[1]}
        results_participant = results_participant.append(this_row, ignore_index = True)
    results = results.append(results_participant, ignore_index = True)

filepath = "../outputs/results_expected_accuracy.csv"
results.to_csv(filepath, index = False)

In [ ]:
results = pd.DataFrame(columns=['condition', 'fsm', 'task', 'beta', 'sum_log_p'])
this_row = {}
for task in ["prediction", "explanation", "control"]:
    for fsm in ["easy", "hard"]:
        for condition in ["visible_beta", "hidden_beta", "hidden_beta_an"]:
            running=np.array(sample[2]["hidden_beta_an"])
            running[:,1] *= 0
            counter = 0
            for i in sample.keys():
                if sample[i]["task"] == task and sample[i]["fsm"] == fsm:
                    counter += 1
                    running[:,1] += np.array(sample[i][condition])[:,1]
            for r in running:
                this_row = {'condition':condition, "fsm": fsm, "task":task, "beta": r[0], "sum_log_p": r[1], "count": counter}
                results = results.append(this_row, ignore_index = True)

filepath = "../outputs/results_mm_running.csv"
results.to_csv(filepath, index = False)

In [ ]:
results = pd.DataFrame(columns=['participant_id','condition', 'fsm', 'task', 'beta', 'sum_log_p'])
this_row = {}
for participant_id in sample.keys():
    results_participant = pd.DataFrame(columns=['participant_id','condition', 'fsm', 'task', 'beta', 'sum_log_p'])
    fsm = sample[participant_id]["fsm"]
    task = sample[participant_id]["task"]
    for condition in ["visible_beta", "hidden_beta", "hidden_beta_an", "visible_beta_fsm", "hidden_beta_fsm", "hidden_beta_fsm_an"]:
        betas = sample[participant_id][condition]
        for row in betas:
            this_row = {"participant_id": participant_id, 'condition':condition, "fsm": fsm, "task":task, "beta": row[0], "sum_log_p": row[1]}
            results_participant = results_participant.append(this_row, ignore_index = True)
    results = results.append(results_participant, ignore_index = True)

filepath = "../outputs/results_mm_participants.csv"
results.to_csv(filepath, index = False)

In [ ]:
sample[2]

In [ ]:
running=np.array(sample[2]["hidden_beta_an"])
running[:,1] *= 0

In [ ]:
betas = sample[participant_id][condition]
fsm = sample[participant_id]["fsm"]
task = sample[participant_id]["task"]
results_participant = pd.DataFrame(columns=['participant_id','condition', 'fsm', 'task', 'beta', 'sum_log_p'])
this_row = {"participant_id": participant_id, 'condition':condition, "fsm": fsm, "task":task, "beta": row[0], "sum_log_p": row[1]}
results_participant = results.append(this_row, ignore_index = True)

In [ ]:
results_participant = pd.DataFrame(columns=['participant_id','condition', 'fsm', 'task', 'beta', 'sum_log_p'])
betas = sample[participant_id][condition]
fsm = sample[participant_id]["fsm"]
task = sample[participant_id]["task"]
for row in betas:
    this_row = {"participant_id": participant_id, 'condition':condition, "fsm": fsm, "task":task, "beta": row[0], "sum_log_p": row[1]}
    results_participant = results_participant.append(this_row, ignore_index = True)

In [ ]:
results_participant